The purpose of this file is to load World bank indicator data, obtained in CSV format, downloading from the [World Bank Indicator portal](http://datatopics.worldbank.org/world-development-indicators/)  

The data is reformated to a pandas dataframe with following format specified: indicators are on columns and the country and year form a multi level row index

In [5]:
import pandas as pd
import json
from pandas_datareader import wb

In [61]:
data_dir = '.\\..\\..\\data\\'
#name of output pickle file
world_bank_file_out = "world_bank_bulk_data.pkl"
#Name of input bulk csv file from the World bank website
world_bank_file_input = "WDIData.csv"
wb_data = pd.read_csv(data_dir + world_bank_file_input)
WDI_name_lookup_output = "WDI_name_lookup.csv"

#### Format as received from the World Bank:

In [7]:
print("Shape:",wb_data.shape)

wb_data.head(3)

Shape: (422136, 64)


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
0,Arab World,ARB,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"2005 PPP conversion factor, private consumptio...",PA.NUS.PRVT.PP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,82.407647,82.827636,83.169227,83.587141,83.954293,84.23063,84.570425,NaN,NaN,NaN


#### Create lookup table (pandas dataframe) of Indicator Names using Indicator Codes

This is currently used by the tableau dashboard (see README.md)

In [65]:
#We only need one each of the values for the indicators. Dropping duplicates will speed up..
#the following code conciderably
wb_data_small = wb_data.drop_duplicates(['Indicator Code', 'Indicator Name'])
name_lookup = pd.DataFrame(columns=['Indicator Code', 'Indicator Name'])
#Fill the indicator column in our lookup table
name_lookup['Indicator Code'] = wb_data_small['Indicator Code'].unique()
#Fill the Indicator Name column of the lokup table
name_lookup['Indicator Name'] = [wb_data_small[wb_data_small['Indicator Code'] == code]['Indicator Name'].values[0] for code in name_lookup['Indicator Code'] ]

#### Create dataframe for further processing

In [7]:
#filter out unnecessary columns (we will focus on years 1972 to 2018)
drop = ['Country Code','Indicator Name','1960','1961','1962','1963','1964',\
        '1965','1966','1967','1968','1969', '1970', '1971', 'Unnamed: 63']
wb_data = wb_data.drop(drop, axis='columns')

In [8]:
#Switch around the data so that indicators are on columns and the country and year form a multi level row index
wb_data = wb_data.set_index(['Country Name', 'Indicator Code'])
wb_data = wb_data.stack()
wb_data = wb_data.unstack(['Indicator Code'])
wb_data = wb_data.sort_index()
wb_data.index.levels[1].name = 'Year'
wb_data.index.levels[0].name = 'Country'

#### Output format:

In [9]:
wb_data.head(10)

Indicator Code    EG.CFT.ACCS.ZS  EG.ELC.ACCS.ZS  ...  NY.GSR.NFCY.KN  \
Country     Year                                  ...                   
Afghanistan 1972             NaN             NaN  ...             NaN   
            1973             NaN             NaN  ...             NaN   
            1974             NaN             NaN  ...             NaN   
            1975             NaN             NaN  ...             NaN   
            1976             NaN             NaN  ...             NaN   
            1977             NaN             NaN  ...             NaN   
            1978             NaN             NaN  ...             NaN   
            1979             NaN             NaN  ...             NaN   
            1980             NaN             NaN  ...             NaN   
            1981             NaN             NaN  ...             NaN   

Indicator Code    SH.STA.FGMS.ZS  
Country     Year                  
Afghanistan 1972             NaN  
            1973             NaN  
            1974             NaN  
            1975             NaN  
            1976             NaN  
            1977             NaN  
            1978             NaN  
            1979             NaN  
            1980             NaN  
            1981             NaN  

[10 rows x 1594 columns]

#### Save to a pickle file:

In [10]:
#Write data to a pickle file
wb_data.to_pickle(data_dir + world_bank_file_out)
#Write lookup data to a CSV file
name_lookup.to_csv(data_dir + WDI_name_lookup_output)